In [32]:
from sqlalchemy import create_engine, text 
import pandas as pd 
import numpy as np 

In [20]:
sql = "SELECT * FROM books"
connection_string = "sqlite:////Users/flint/Data/kaggle/books/book-recommender/booksdb2.sql"

In [21]:
engine = create_engine(connection_string)
conn = engine.connect()
books = pd.read_sql(text(sql), con=conn)
conn.close()

In [22]:
books.head(2)

,book_id_mapping,title,num_pages,name
0,808,Pride and Prejudice and Zombies Pride and Prej...,320,Jane Austen
1,217,Blue Bloods Blue Bloods 1,302,Melissa de la Cruz


## Raccomandazione
Voglio raccomandare a un utente libri che non ha ancora letto (votato) ma che sono stati letti da utenti simili a lui
1. Criterio di similarità fra utenti
2. (Clustering)
3. Preso un utente $u$:
   1. Estraggo i $k$ utenti più simili (K)
   2. L'insieme dei libri letti (votati) da K
   3. Determino l'insieme $U$ dei libri letti da $u$ e calcolo $C = K - U$
   4. Faccio un ranking di $C$

### Similarità fra utenti
- Due utenti sono simili se danno voti simili alle stesse **cose** 
- Dove **cose** può riferirsi a:
  - libri
  - <u>generi</u>
  - autori

### Task 1: rappresentare le preferenze degli utenti per i generi letterari

In [23]:
user_profile = """
SELECT R.user_id_mapping, G.genre,
    AVG(R."Actual Rating") AS score
FROM genre AS G JOIN ratings AS R
ON G.book = R.book_id_mapping
GROUP BY user_id_mapping, genre
"""

In [24]:
conn = engine.connect()
profiles = pd.read_sql(text(user_profile), con=conn)
conn.close()

In [34]:
genre_list = list(profiles.genre.unique())
n = len(profiles.genre.unique())
n, genre_list

(10,
 ['fantasy, paranormal',
  'fiction',
  'romance',
  'young-adult',
  'children',
  'comics, graphic',
  'history, historical fiction, biography',
  'non-fiction',
  'mystery, thriller, crime',
  'poetry'])

In [48]:
u2 = np.zeros(n)
u5 = np.zeros(n)

In [49]:
u2p = profiles[profiles['user_id_mapping'] == 2][['genre', 'score']]
u5p = profiles[profiles['user_id_mapping'] == 55][['genre', 'score']]

In [50]:
for i, row in u2p.iterrows():
    g, s = row['genre'], row['score']
    i_g = genre_list.index(g)
    u2[i_g] = s 
for i, row in u5p.iterrows():
    g, s = row['genre'], row['score']
    i_g = genre_list.index(g)
    u5[i_g] = s 

In [56]:
np.sqrt(np.power(u2 - u5, 2).sum())

5.787918451395113

In [29]:
profiles[profiles['user_id_mapping'] == 55][['genre', 'score']]

,genre,score
36,children,5.0
37,"fantasy, paranormal",2.0
38,fiction,3.5
39,romance,2.0
40,young-adult,3.5
